# core

> Helper functions and classes for Conc.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# requirements - numpy pandas polars spacy nltk great_tables
# dev requirements - nbdev, jupyterlab, memory_profiler
# TODO check all necessary

import re
import os
import logging
import polars as pl
from great_tables import GT

In [ ]:
#| export
PAGE_SIZE = 20
EOF_TOKEN_STR = ' conc-end-of-file-token'

In [ ]:
#| exporti
class ConcLogger(logging.Logger):
	""" Custom logger for conc module. """
	def __init__(self, name, level=logging.WARNING):
		super().__init__(name, level)
		self._setup_handler()

	def _setup_handler(self):
		handler = logging.StreamHandler()
		formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(funcName)s - %(message)s', 
									  datefmt='%Y-%m-%d %H:%M:%S')
		handler.setFormatter(formatter)
		self.addHandler(handler)

	def set_state(self, state:str # 'quiet' or 'verbose'
				  ):
		if state == 'quiet':
			level = logging.WARNING
		elif state == 'verbose':
			level = logging.INFO
		else:
			raise ValueError(f"Invalid state: {state}")
		
		self.setLevel(level)



In [ ]:
#| export
logging.setLoggerClass(ConcLogger)

logger = logging.getLogger(__name__)

In [ ]:
#| export
def set_logger_state(state:str # 'quiet' or 'verbose'
					 ):
	""" Set the state of the conc logger to either 'quiet' or 'verbose' """
	logger.set_state(state)

In [ ]:
#| exporti

toy_data = []
toy_data.append(['1.txt', 'The cat sat on the mat.', 'feline', 'cat'])
toy_data.append(['2.txt', 'The dog sat on the mat.', 'canine', 'dog'])
toy_data.append(['3.txt', 'The cat is meowing.', 'feline', 'cat'])
toy_data.append(['4.txt', 'The dog is barking.', 'canine', 'dog'])
toy_data.append(['5.txt', 'The cat is climbing a tree.', 'feline', 'cat'])
toy_data.append(['6.txt', 'The dog is digging a hole.', 'canine', 'dog'])

source_path = '../test-corpora/source'

In [ ]:
#| export
def create_toy_corpus_sources(source_path:str # patch to location of sources for building corpora
							 ):
	""" Create txt files and csv to test build of toy corpus. """

	toy_path = os.path.join(source_path, 'toy')
	if not os.path.exists(toy_path):
		os.makedirs(toy_path, exist_ok=True)
	for row in toy_data:
		with open(os.path.join(source_path, 'toy', row[0]), 'w', encoding='utf-8') as f:
			f.write(row[1])
	df = pl.DataFrame(toy_data, orient='row', schema=(('source', str), ('text', str), ('category', str), ('species', str)))
	df.write_csv(os.path.join(source_path, 'toy.csv'))
	df.write_csv(os.path.join(source_path, 'toy.csv.gz'))


In [ ]:
#| hide
#| eval: false
create_toy_corpus_sources(source_path)

In [ ]:
#| hide
toy_corpus_df = pl.read_csv(os.path.join(source_path, 'toy.csv'))
GT(toy_corpus_df).tab_options(table_margin_left = 0).show()

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| export
def get_nltk_corpus_sources(source_path:str # patch to location of sources for building corpora
							 ):
	""" Get nltk corpora as sources for testing. """

	import nltk
	nltk.download('gutenberg')
	nltk.download('brown')
	nltk.download('reuters')
	from nltk.corpus import gutenberg
	from nltk.corpus import reuters
	from nltk.corpus import brown

	def clean_text(text):
		# to match words/punc that followed by /tags
		pattern = re.compile(r"(\S+)(/[^ ]+)") # match non-space followed by / and non-space
		return pattern.sub(r"\1", text)

	brown_path = os.path.join(source_path, 'brown.csv.gz')
	corpus_data = []
	for fileid in brown.fileids():
		corpus_data.append([fileid, clean_text(brown.raw(fileid))])
		with open(os.path.join(source_path, 'brown', f'{fileid}.txt'), 'w', encoding='utf-8') as f:
			f.write(clean_text(brown.raw(fileid)))
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(brown_path)

	gutenberg_path = os.path.join(source_path, 'gutenberg.csv.gz')
	corpus_data = []
	for fileid in gutenberg.fileids():
		corpus_data.append([fileid, clean_text(gutenberg.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(gutenberg_path)

	reuters_path = os.path.join(source_path, 'reuters.csv.gz')
	corpus_data = []
	for fileid in reuters.fileids():
		fileid_name = fileid.split('/')[1]
		corpus_data.append([fileid_name, clean_text(reuters.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(reuters_path)


In [ ]:
#| hide
#| eval: false
get_nltk_corpus_sources(source_path)

[nltk_data] Downloading package gutenberg to /home/geoff/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package brown to /home/geoff/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /home/geoff/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


The texts for the Brown corpus from nltk can be used for testing Conc functionality. The Reuters and Gutenberg corpora are also prepared by `get_nltk_corpus_sources`. Running the function will download the texts and save the texts as a .csv.gz files with columns: source and text. The Brown Corpus is also saved as .txt files to test the Corpus.build_from_texts method.

In [ ]:
#|hide
# TODO - cleanup or move to function - creating sample data from large data-set
# create_sample_data = False
# if create_sample_data == True:
# 	# load standard csv library
# 	import csv
# 	import gzip

# 	for max_i in [10000, 100000, 200000, 500000]:
# 		max_i_label = int(max_i / 1000)
# 		# create version with just first 100000 rows
# 		with gzip.open('../test-corpora/source/rnz.csv.gz', 'rt') as f:
# 			reader = csv.DictReader(f)
# 			with gzip.open(f'../test-corpora/source/rnz-{max_i_label}k.csv.gz', 'wt') as f_out:
# 				writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames)
# 				writer.writeheader()
# 				for i, row in enumerate(reader):
# 					if i > max_i - 1:
# 						break
# 					writer.writerow(row)
# 			print(f'Created file rnz-{max_i_label}k.csv.gz')


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()